# Causal Inference
## Задание CI.5


**Правила:**

* Дедлайны см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Выполненную работу нужно отправить телеграм-боту `@miptstats_ds24_bot`. Для начала работы с ботом каждый раз отправляйте `/start`. **Работы, присланные иным способом, не принимаются.**
* Дедлайны см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно **ноутбук в формате `ipynb`**.
* Следите за размером файлов. **Бот не может принимать файлы весом более 20 Мб.** Если файл получается больше, заранее разделите его на несколько.
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания будут сдавать устный зачет.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*
* В каждой задаче не забывайте делать **пояснения и выводы**.

**Правила оформления теоретических задач:**

* Решения необходимо прислать одним из следующих способов:
  * фотографией в правильной ориентации, где все четко видно, а почерк разборчив,
    * отправив ее как файл боту вместе с ноутбуком *или*
    * вставив ее в ноутбук посредством `Edit -> Insert Image` (<font color="red">фото, вставленные ссылкой, не принимаются</font>);
  * в виде $\LaTeX$ в markdown-ячейках.
* Решения не проверяются, если какое-то требование не выполнено. Особенно внимательно все проверьте в случае выбора второго пункта (вставки фото в ноутбук). <font color="red"><b>Неправильно вставленные фотографии могут не передаться при отправке.</b></font> Для проверки попробуйте переместить `ipynb` в другую папку и открыть его там.
* В решениях поясняйте, чем вы пользуетесь, хотя бы кратко. Например, если пользуетесь независимостью, то достаточно подписи вида "*X и Y незав.*"
* Решение, в котором есть только ответ, и отсутствуют вычисления, оценивается в 0 баллов.

---


**Баллы за задание:**

* Задача 1 — 100 баллов
* Задача 2 — 100 баллов

In [1]:
# Bot check

# HW_ID: ds_ci5
# Бот проверит этот ID и предупредит, если случайно сдать что-то не то.

# Status: final
# Перед отправкой в финальном решении удали "not" в строчке выше.
# Так бот проверит, что ты отправляешь финальную версию, а не промежуточную.
# Никакие значения в этой ячейке не влияют на факт сдачи работы.

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error as MSE, mean_absolute_error as MAE
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'torch'

---
### Задача 1.

Напишите реализацию нейронной сети DragonNet с регуляризацией. Предусмотрите три варианта лосса:
* простой без регуляризации,
* с регуляризацией,
* в версии BCAUSS.

In [ ]:
class DragonNet(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(DragonNet, self).__init__()
        self.fc1 = nn.Linear(input_dim,hidden_dim)
        self.fc2 = nn.Linear(hidden_dim,hidden_dim)
        self.fc3 = nn.Linear(hidden_dim,1)
        self.fc4 = nn.Linear(hidden_dim,1)
    def forward(self, x, t):
        h = torch.relu(self.fc2(torch.relu(self.fc1(x))))
        return self.fc3(h), torch.sigmoid(self.fc4(h))  #q,g

def dragonnet_loss(model, x, t, y, alpha=0, beta=0):
    q, g = model(x, t)
    q_t = q*t + (1-t)*(1-q)
    return torch.mean((q_t-y)**2) + alpha*nn.BCELoss()(g,t.float()) + beta*torch.mean((q_t-y)**2 * (t/g - (1-t)/(1-g)))

def train_dragonnet(x, t, y, alpha, beta, epochs=1000):
    model = DragonNet(x.shape[1], 64)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for _ in range(epochs):
        optimizer.zero_grad()
        loss = dragonnet_loss(model, x, t, y, alpha, beta)
        loss.backward()
        optimizer.step()
    return model

Для проверки модели возьмем полусинтетические данные IHDP (Infant Health and Development Program), которые можно скачать по ссылкам: <a href="https://www.fredjo.com/files/ihdp_npci_1-1000.train.npz.zip">трейн</a>, <a href="https://www.fredjo.com/files/ihdp_npci_1-1000.test.npz.zip">тест</a>.

Каждый файл является numpy-архивом, который считывается следующим образом

In [ ]:
data = np.load('ihdp_npci_1-1000.train.npz')
data.files

Значения файлов:
* `x` &mdash; признаки (ковариаты, конфаундеры);
* `t` &mdash; тритмент;
* `yf` &mdash; фактические исходы (таргет);
* `ycf` &mdash; контрафактивные исходы;
* `mu0, mu1` &mdash; потенциальные исходы до зашумления.

Значения каждого отдельного файла можно получить, вызвав его по имени подобно словарю.

Например, вот истинный теоретический причинно-следственный эффект

In [ ]:
data['ate']

Обучающий набор содержит 1000 датасетов по 672 объекта в каждом

In [ ]:
data['x'].shape, data['t'].shape, data['yf'].shape

Посчитайте истинную ATE отдельно для каждого датасета. Это можно сделать потому как в этом синтетическом датасете мы знаем контрафактивные исходы. Далее будем сравнивать оценку модели именно с этими значениями, а не с теоретическим значением `data['ate']`.

In [ ]:
true_ate = np.mean(data['mu1']-data['mu0'], axis=1)
print(f'Истинная АТЕ: {true_ate}')

Для каждого из 1000 датасетов 
* обучите модель с регуляризацией и без нее,
* обучите модель с лоссом BCAUSS,
* посчитайте оценки ATE.

По результатам посчитайте метрики RMSE и MAE.

In [ ]:
def estimate_ate(model, x):
    return torch.mean(model(torch.tensor(x).float(), torch.ones(x.shape[0]))[0] - model(torch.tensor(x).float(), torch.zeros(x.shape[0]))[0])

ks = ['ate', 'reg_ate', 'bcauss_ate']
logs = {k:[] for k in ks}
for i in tqdm(range(1000)):
    for k,a,b in zip(ks,[0,1,1],[0,0,1]):
        logs[k].append(estimate_ate(train_dragonnet(data['x'][i], data['t'][i], data['yf'][i], alpha=a, beta=b), data['x'][i]).item())

display(pd.DataFrame({'XP':ks, 'RMSE':[np.sqrt(MSE(true_ate, logs[k])) for k in ks], 'MAE':[MAE(true_ate, logs[k]) for k in ks]}).round(3))

Для некоторых датасетов сравните визуально гистограммы propensity score и balancing score.

In [ ]:
plt.figure(figsize=(15,7))
for j, i in enumerate((np.random.random(5)*1000).astype(int)):
    P=LogisticRegression().fit(data['x'][...,i], data['t'][:,i]).predict_proba(data['x'][...,i])[:,1]
    ax = plt.subplot(2, 3, j+1)
    plt.hist(P[np.argwhere(data['t'][:,i]==0)], label='0', alpha=0.3)
    plt.hist(P[np.argwhere(data['t'][:,i]==1)], label='1', alpha=0.3)
    plt.legend()
    plt.xlabel('score')
plt.show()

Проведите аналогичные действия с моделями
* линейная регрессия,
* propensity score weighting,
* doubly robust estimation.

Можно использовать как реализации из обучающих ноутбуков, так и библиотечные.

Сравните результат с DragonNet.

In [ ]:
ks = ['lr_ate', 'ps_ate', 'dr_ate']
logs = {k:[] for k in ks}
for i in tqdm(range(1000)):
    lr = LinearRegression().fit(data['x'][i], data['y'][i])
    logs['lr_ate'].append(lr.predict(data['x'][i]).mean())
    ps = np.mean(data['t'][i])
    logs['ps_ate'].append(np.mean(data['y'][i] * (data['t'][i] / ps + (1-data['t'][i]) / (1-ps))))
    logs['dr_ate'].append(np.mean(data['y'][i] - lr.predict(data['x'][i]) + data['t'][i] * (data['y'][i] - lr.predict(data['x'][i])) / ps))

display(pd.DataFrame({'XP':ks, 'RMSE':[np.sqrt(MSE(true_ate, logs[k])) for k in ks], 'MAE':[MAE(true_ate, logs[k]) for k in ks]}).round(3))

Сделайте выводы.

---
### Задача 2.

На занятии мы посмотрели работу простых моделей на данных <a href="https://ods.ai/competitions/x5-retailhero-uplift-modeling">конкурса X5 Retail Hero</a>. Напомним, требуется построить модель, определяющую клиентов, которым стоит отправлять смс. Отметим, что в данном случае нам не требуется как можно точнее оценить CATE, требуется скорее определить степерь уверенности того, что клиент совершит покупку из-за получения смс, или наоборот.

Стандартные способы загрузки данных:
1. непосредственно со <a href="https://ods.ai/competitions/x5-retailhero-uplift-modeling">страницы конкура</a>;
2. вызовом <a href="https://www.uplift-modeling.com/en/latest/api/datasets/fetch_x5.html">`data = sklift.datasets.fetch_x5()`</a>

**Но прежде всего обратите внимание, что данные большие, и все целиком они могут не вместиться даже в достаточно большую оперативку, например, в Colab не помещаются.** Поэтому мы подготовили для вас сжатый формат данных, которые вы можете скачать по ссылкам ниже.
1. Данные о <a href="https://disk.yandex.ru/d/ZxtypOGcVm9ibw">клиентах</a>, <a href="https://disk.yandex.ru/d/b9czlSQeLAU0dw">тритменте и таргете</a>, все в исходном формате.
2. <a href="https://disk.yandex.ru/d/BvDtDzV_4rJfDg">Сгруппированные данные по покупкам</a>, то есть для каждой покупки оставляется только информация о сумме покупке, количеству уникальных позиций.
3. Те же сгруппированные данные, но <a href="https://disk.yandex.ru/d/gQn1zn6PBHkzXA">разделенные случайно на 10 частей</a>.
3. Полные данные, <a href="https://disk.yandex.ru/d/gDFMnY2PkpsX3g">разделенные случайно на 50 частей</a>. Каждый клиент присутствует только в одной части.

*Рекомендация.* Начинайте с небольших данных, для отладки кода выделяйте небольшое количество клиентов. Переходите к большему объему данных только после того, как построете хорошие модели на меньшем объеме. Сохраняйте рабочие версии на случай если что-то случайно испортить.

Постройте различные модели:
* S-, T-, X-, R-learner;
* Трансформация таргета;
* Многоклассовая модель;
* DragonNet, которую вы реализовали в задаче 1.

Все эти модели расписаны в ноутбуке с занятия.
Для построения моделей обязательно нужно использовать данные о покупках. Не обязательно полные, например, можно посчитать сумму покупок клиента в месяц, его средний чек, количество покупок в месяц, и т.д.. Количество используемых признаков скорее зависит от возможностей ваших ресурсов.

Постройте для этих моделей кривые Qini и Uplift, а также оцените метрику `uplift@30%`. На их основе выберите одну модель.

Для выбранной модели постройте гистограммы предсказаний потенциальных исходов (если модель это позволяет) и значений uplift (или оценок CATE). Постройте также графики `plot_uplift_by_percentile`.

Сделайте выводы.